In [43]:
%matplotlib inline
import pandas as pd
import numpy as np
import re
from matplotlib.ticker import MaxNLocator
import matplotlib.pyplot as plt
from requests import get
from bs4 import BeautifulSoup
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [44]:
DATA_PATH = "data/"
AFDB_2019 = "afdb_1960_2019/Data.csv"
db_zero = pd.read_csv(DATA_PATH + AFDB_2019, sep=',')
db_zero = db_zero.dropna()
db_zero.head()

,CountryName,Country,IndicatorName,Indicator,Units,Scale,Frequency,Date,Value
0,AFRICA,AFR,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,annual %,1,A,2000-01-01,9.133374
1,AFRICA,AFR,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,annual %,1,A,2001-01-01,8.535367
2,AFRICA,AFR,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,annual %,1,A,2002-01-01,7.238168
3,AFRICA,AFR,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,annual %,1,A,2003-01-01,7.217622
4,AFRICA,AFR,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,annual %,1,A,2004-01-01,7.373657


In [45]:
db_zero.Date = db_zero.Date.apply(lambda x : x.split("-")[0]).astype(int)
db_zero.rename(columns = {'Date':'Year'}, inplace = True)
db_zero.Scale = db_zero.Scale.astype(int)
db_zero.Value = db_zero.Value.astype(float)
db_zero.head()

,CountryName,Country,IndicatorName,Indicator,Units,Scale,Frequency,Year,Value
0,AFRICA,AFR,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,annual %,1,A,2000,9.133374
1,AFRICA,AFR,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,annual %,1,A,2001,8.535367
2,AFRICA,AFR,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,annual %,1,A,2002,7.238168
3,AFRICA,AFR,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,annual %,1,A,2003,7.217622
4,AFRICA,AFR,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,annual %,1,A,2004,7.373657


In [49]:
db = db_zero[['Country','Year','Indicator','Value']].set_index(['Country', 'Year'])

db = pd.pivot_table(db_zero, values='Value', index=['Country', 'Year'], columns=['Indicator'])
db

Indicator     AG.CCL.FRST.MT  AG.CON.FERT.MT  AG.EXPO.CREL.MT  AG.FWC.FRST.M3  \
Country Year                                                                    
ADB     1960             NaN             NaN              NaN             NaN   
        1961             NaN             NaN              NaN             NaN   
        1962             NaN             NaN              NaN             NaN   
        1963             NaN             NaN              NaN             NaN   
        1964             NaN             NaN              NaN             NaN   
...                      ...             ...              ...             ...   
ZZM     2046             NaN             NaN              NaN             NaN   
        2047             NaN             NaN              NaN             NaN   
        2048             NaN             NaN              NaN             NaN   
        2049             NaN             NaN              NaN             NaN   
        2050             NaN             NaN              NaN             NaN   

Indicator     AG.FWN.FRST.M3  AG.IMP.CREL.MT  AG.IMP.FERT.CD  AG.LND.AGRI.HA  \
Country Year                                                                   
ADB     1960             NaN             NaN             NaN             NaN   
        1961             NaN             NaN             NaN             NaN   
        1962             NaN             NaN             NaN             NaN   
        1963             NaN             NaN             NaN             NaN   
        1964             NaN             NaN             NaN             NaN   
...                      ...             ...             ...             ...   
ZZM     2046             NaN             NaN             NaN             NaN   
        2047             NaN             NaN             NaN             NaN   
        2048             NaN             NaN             NaN             NaN   
        2049             NaN             NaN             NaN             NaN   
        2050             NaN             NaN             NaN             NaN   

Indicator     AG.LND.AGRI.ZS  AG.LND.ARBL.HA  ...  TG.WEO.TXGO.CD  \
Country Year                                  ...                   
ADB     1960             NaN             NaN  ...             NaN   
        1961             NaN             NaN  ...             NaN   
        1962             NaN             NaN  ...             NaN   
        1963             NaN             NaN  ...             NaN   
        1964             NaN             NaN  ...             NaN   
...                      ...             ...  ...             ...   
ZZM     2046             NaN             NaN  ...             NaN   
        2047             NaN             NaN  ...             NaN   
        2048             NaN             NaN  ...             NaN   
        2049             NaN             NaN  ...             NaN   
        2050             NaN             NaN  ...             NaN   

Indicator     TG.WEO.TXGO_R.KD  TG.WEO.TXGXO.CD  TG.WEO.TXGXO_R.KD  \
Country Year                                                         
ADB     1960               NaN              NaN                NaN   
        1961               NaN              NaN                NaN   
        1962               NaN              NaN                NaN   
        1963               NaN              NaN                NaN   
        1964               NaN              NaN                NaN   
...                        ...              ...                ...   
ZZM     2046               NaN              NaN                NaN   
        2047               NaN              NaN                NaN   
        2048               NaN              NaN                NaN   
        2049               NaN              NaN                NaN   
        2050               NaN              NaN                NaN   

Indicator     TG.WEO.TXG_R.KD  TG.WEO.TXG_R.ZG  TG.WEO.TXS.CD  TG.WEO.TX_R.KD  \
Country Year   

In [70]:
db.loc['TUN',2010 or 'TUN',2011]

TypeError: cannot do label indexing on <class 'pandas.core.indexes.base.Index'> with these indexers [2010] of <class 'int'>

In [67]:
db.dropna()

,Indicator,AG.CCL.FRST.MT,AG.CON.FERT.MT,AG.EXPO.CREL.MT,AG.FWC.FRST.M3,AG.FWN.FRST.M3,AG.IMP.CREL.MT,AG.IMP.FERT.CD,AG.LND.AGRI.HA,AG.LND.AGRI.ZS,AG.LND.ARBL.HA,...,TG.WEO.TXGO.CD,TG.WEO.TXGO_R.KD,TG.WEO.TXGXO.CD,TG.WEO.TXGXO_R.KD,TG.WEO.TXG_R.KD,TG.WEO.TXG_R.ZG,TG.WEO.TXS.CD,TG.WEO.TX_R.KD,TG.WEO.TX_R.ZG,VC.PRS.UNSEC
Country,Year,,,,,,,,,,,,,,,,,,,,,
